In [1]:
import hashlib
import datetime

class Block:
    def __init__(self, message, previous_hash=''):
        self.timestamp = str(datetime.datetime.now())
        self.message = message
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        data = self.timestamp + self.message + self.previous_hash
        return hashlib.sha256(data.encode()).hexdigest()

    def __str__(self):
        return f"Message: {self.message}\nTime: {self.timestamp}\nPrev Hash: {self.previous_hash}\nHash: {self.hash}\n"

# Tạo chuỗi Blockchain
blockchain = []
blockchain.append(Block("Khối đầu tiên"))
blockchain.append(Block("Khối thứ hai", blockchain[-1].hash))
blockchain.append(Block("Khối thứ ba", blockchain[-1].hash))

# In ra thông tin các khối
for i, block in enumerate(blockchain):
    print(f"Block {i+1}:\n{block}")

# Thử thay đổi dữ liệu ở khối thứ 2
print(">> Thay đổi thông điệp trong khối thứ hai:")
blockchain[1].message = "Đã bị sửa đổi"
blockchain[1].hash = blockchain[1].calculate_hash()
print(blockchain[1])


Block 1:
Message: Khối đầu tiên
Time: 2025-04-23 07:53:41.024943
Prev Hash: 
Hash: b4bfb86ffa15efb70908110c0436c1da11b6293a4af23972eac80423a0da2108

Block 2:
Message: Khối thứ hai
Time: 2025-04-23 07:53:41.024943
Prev Hash: b4bfb86ffa15efb70908110c0436c1da11b6293a4af23972eac80423a0da2108
Hash: 9b72077a54a666db1c5df570339fff1b9eee6885d7220c5a8b779e54351c56df

Block 3:
Message: Khối thứ ba
Time: 2025-04-23 07:53:41.024943
Prev Hash: 9b72077a54a666db1c5df570339fff1b9eee6885d7220c5a8b779e54351c56df
Hash: 51cd27285e6c2c013cf491685919ea608f9f6242818563b22b0ec76796bfe936

>> Thay đổi thông điệp trong khối thứ hai:
Message: Đã bị sửa đổi
Time: 2025-04-23 07:53:41.024943
Prev Hash: b4bfb86ffa15efb70908110c0436c1da11b6293a4af23972eac80423a0da2108
Hash: 1bdfa03a7066681e99e0937f60209180eda47e84f3f231e934f0f8a4e354b1d9



In [2]:
import hashlib

def sha256_hash(text):
    return hashlib.sha256(text.encode()).hexdigest()

message1 = "Học phần Mạng máy tính"
message2 = "Học phần mạng máy tính"  # đổi chữ "M" thành "m"

hash1 = sha256_hash(message1)
hash2 = sha256_hash(message2)

print(f"Chuỗi 1: {message1}\nHash: {hash1}")
print(f"Chuỗi 2: {message2}\nHash: {hash2}")
print("\nNhận xét: Thay đổi chỉ 1 ký tự → mã băm thay đổi hoàn toàn.")


Chuỗi 1: Học phần Mạng máy tính
Hash: 65c684ecb5b1be63ed30fed125ee8ac1842cdc1b87f32a0772309f174608b2b2
Chuỗi 2: Học phần mạng máy tính
Hash: 1e357c2d41854f79edcc70d4ee1c567ab9f14e3cc67eb25feda6b7652d49c42b

Nhận xét: Thay đổi chỉ 1 ký tự → mã băm thay đổi hoàn toàn.


In [ ]:
from web3 import Web3
import solcx
import json

# Kết nối Ganache
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
assert w3.is_connected(), "Không kết nối được Ganache!"

# Tài khoản từ Ganache
account = w3.eth.accounts[0]

# Đọc hợp đồng Solidity
with open("Message.sol", "r") as file:
    source = file.read()

# Biên dịch
compiled = solcx.compile_source(source)
contract_id, contract_interface = compiled.popitem()

# Triển khai
Message = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])
tx_hash = Message.constructor().transact({'from': account})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
contract_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=contract_interface['abi'])

# Gửi thông điệp
tx = contract_instance.functions.setMessage("Xin chào Blockchain!").transact({'from': account})
w3.eth.wait_for_transaction_receipt(tx)

# Đọc lại thông điệp
message = contract_instance.functions.getMessage().call()
print("Thông điệp từ Smart Contract:", message)


AssertionError: Không kết nối được Ganache!